# 구글 드라이브 연결후 데이터 짚파일 열기
# 데이터 열어보기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_path = '/content/drive/MyDrive/data.zip'


In [ ]:
import zipfile
import os


extract_dir = '/content/drive/MyDrive/Boostup2025/'


os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("압축 해제 완료:", extract_dir)



압축 해제 완료: /content/drive/MyDrive/Boostup2025/


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Boostup2025/train.csv')

#데이터 확인 결과
- ID/Canonical_Smiles : object
- Inhibition : float64
목적: Canonical_Smiles라는 화학 구조 표현을 머신러닝에 사용할 수 있는 벡터로 변환하고, 이를 통해 화합물의 억제율(Inhibition) 을 예측하는 회귀 모델을 학습함

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1681 entries, 0 to 1680
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                1681 non-null   object 
 1   Canonical_Smiles  1681 non-null   object 
 2   Inhibition        1681 non-null   float64
dtypes: float64(1), object(2)
memory usage: 39.5+ KB


# Canonical_Smiles : 화합물 구조로 되어 있음


In [ ]:
df.head(10)

,ID,Canonical_Smiles,Inhibition
0,TRAIN_0000,Cl.OC1(Cc2cccc(Br)c2)CCNCC1,12.50
1,TRAIN_0001,Brc1ccc2OCCc3ccnc1c23,4.45
2,TRAIN_0002,CC1(CO)CC(=NO1)c2cc(c(F)cc2Cl)[N+](=O)[O-],4.92
3,TRAIN_0003,Fc1ccc2nc(Nc3cccc(COc4cccc(c4)C(=O)N5CCOCC5)c3...,71.50
4,TRAIN_0004,CC(C)CC(=O)C1=C(Nc2c(Cl)ccc(Cl)c2C1=O)S(=O)C,18.30
5,TRAIN_0005,O\C(=C\1/OCCN(Cc2ccccc2)C1=O)\C3CCOCC3,1.28
6,TRAIN_0006,CS(=O)(=O)OCC(COCc1c(F)cccc1F)OS(=O)(=O)C,8.18
7,TRAIN_0007,[O-][N+](=O)c1cn2C[C@@H](COc2n1)OC3CCCCO3,13.20
8,TRAIN_0008,Cc1cc(nc(Nc2ccc(C#N)c(c2)C(F)(F)F)n1)N3CCC(CC3...,64.90
9,TRAIN_0009,COC(=O)c1c(C)nc2N(C)CCc2c1OS(=O)(=O)C(F)(F)F,53.50


In [ ]:
df.isnull().sum()

,0
ID,0
Canonical_Smiles,0
Inhibition,0


#화학 구조 → 수치형 벡터 
- RDKit 라이브러리 활용
- Canonical_Smiles → Morgan Fingerprint (2048차원 이진 벡터) 로 변환

In [ ]:
!pip install rdkit-pypi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 76.5 MB/s eta 0:00:00


In [ ]:
# NumPy 1.26.4로 다운그레이드
!pip install numpy==1.26.4 --force-reinstall


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 97.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

# SMILES → Morgan Fingerprint (2048차원 벡터)로 변환
def smiles_to_morgan(smiles, radius=2, n_bits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        return np.array(fp)
    else:
        return np.zeros(n_bits)

# 전체 데이터 변환
X = np.array([smiles_to_morgan(smi) for smi in df['Canonical_Smiles']])
y = df['Inhibition'].values


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
!pip install rdkit-pypi xgboost scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# 1. 데이터 로드 (csv에 헤더가 있다면 header=None 제거)
df = pd.read_csv('/content/drive/MyDrive/Boostup2025/train.csv')

# 만약 컬럼명이 없거나 임의로 지정할 필요 있다면 아래 주석 해제
# df.columns = ['ID', 'Canonical_Smiles', 'Inhibition']

# 2. SMILES → Morgan Fingerprint 변환 함수 (명확한 0/1 배열 반환)
def smiles_to_morgan(smiles, radius=2, n_bits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        bitstring = fp.ToBitString()
        return np.array([int(bit) for bit in bitstring])
    else:
        return np.zeros(n_bits, dtype=int)

X = np.array([smiles_to_morgan(smi) for smi in df['Canonical_Smiles']])
y = df['Inhibition'].values

# 3. 학습/검증 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. RandomForest 모델 학습
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)

# 5. XGBoost 모델 학습
xgb = XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_val)

# 6. 성능 평가 함수
def evaluate(y_true, y_pred, model_name):
    print(f'\n📊 [{model_name}]')
    print(f'MSE : {mean_squared_error(y_true, y_pred):.4f}')
    print(f'R²  : {r2_score(y_true, y_pred):.4f}')

# 7. 결과 출력
evaluate(y_val, rf_pred, 'Random Forest')
evaluate(y_val, xgb_pred, 'XGBoost')




📊 [Random Forest]
MSE : 603.1389
R²  : 0.1320

📊 [XGBoost]
MSE : 629.0826
R²  : 0.0947


1. 성능이 떨어짐
 - random forest : 결정개수가 0.13으로 낮음 ->모델이 평균보다도 못한 예측을 하고 있음
                 : MSE 603으로 정확히 예측 못함
 - XGBoost : 결정개수가 0.0947로 낮음 ->모델이 평균보다도 못한 예측 하고있음
           : MSE 603으로 정확히 예측 못함
2. 이상치 제거 후 돌릴 예정

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 학습된 모델로 전체 데이터 예측 (또는 검증 데이터)
pred_all = rf.predict(X)  # 또는 검증 데이터 X_val

# 잔차 계산
residuals = y - pred_all
abs_residuals = np.abs(residuals)

# 이상치 기준 (예: 잔차 절대값이 2*std 이상인 데이터)
threshold = 2 * np.std(abs_residuals)
outliers_idx = np.where(abs_residuals > threshold)[0]

print(f'이상치로 판단된 샘플 수: {len(outliers_idx)}')

# 이상치 제거한 데이터셋
X_clean = np.delete(X, outliers_idx, axis=0)
y_clean = np.delete(y, outliers_idx, axis=0)


이상치로 판단된 샘플 수: 226


이상치 발견 : 226개 
-> 이상치 제거하기로 결정

In [ ]:
print(f'원본 데이터 개수: {len(y)}')
print(f'이상치 제거 후 데이터 개수: {len(y_clean)}')

# 재학습
X_train2, X_val2, y_train2, y_val2 = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)
rf2 = RandomForestRegressor(n_estimators=200, random_state=42)
rf2.fit(X_train2, y_train2)
rf2_pred = rf2.predict(X_val2)

# 평가
evaluate(y_val2, rf2_pred, 'Random Forest (이상치 제거 후)')


원본 데이터 개수: 1681
이상치 제거 후 데이터 개수: 1455

📊 [Random Forest (이상치 제거 후)]
MSE : 464.6703
R²  : 0.1350


이상치 제거후에도 성능이 나오지 않음 
 - MSE 값의 성능은 좋아졌지만 여전히 결정계수의 값이 낮음
-> 데이터 증강(3회)해보기로 결정

In [ ]:
from rdkit import Chem
from rdkit.Chem import MolToSmiles
import random
import pandas as pd
import numpy as np

def random_smiles(smiles, random_seed=None):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    if random_seed is not None:
        random.seed(random_seed)
    return MolToSmiles(mol, doRandom=True)

#데이터프레임 df에서 증강
augmented_smiles_list = []
augmented_targets = []

for smi, target in zip(df['Canonical_Smiles'], df['Inhibition']):
    augmented_smiles_list.append(smi)
    augmented_targets.append(target)
    for i in range(3):  # 증강 횟수: 3번
        aug_smi = random_smiles(smi, random_seed=i)
        if aug_smi:
            augmented_smiles_list.append(aug_smi)
            augmented_targets.append(target)

aug_df = pd.DataFrame({
    'Canonical_Smiles': augmented_smiles_list,
    'Inhibition': augmented_targets
})

print(f"증강 후 데이터 개수: {len(aug_df)}")


증강 후 데이터 개수: 6724


In [ ]:
# Morgan fingerprint 변환 
def smiles_to_morgan(smiles, radius=2, n_bits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        bitstring = fp.ToBitString()
        return np.array([int(bit) for bit in bitstring])
    else:
        return np.zeros(n_bits, dtype=int)

X_aug = np.array([smiles_to_morgan(smi) for smi in aug_df['Canonical_Smiles']])
y_aug = aug_df['Inhibition'].values

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_aug, y_aug, test_size=0.2, random_state=42)


In [ ]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 23.6 MB/s eta 0:00:00


In [ ]:
import logging
optuna.logging.set_verbosity(logging.INFO)


In [ ]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import optuna.logging

# 로그 레벨 INFO로 설정 (진행 상황 표시 강화)
optuna.logging.set_verbosity(optuna.logging.INFO)

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    max_depth = trial.suggest_int('max_depth', 5, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    rf = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )

    scores = cross_val_score(rf, X_train, y_train, cv=3, scoring='neg_mean_squared_error')
    return -scores.mean()

def print_progress(study, trial):
    print(f"Trial {trial.number}: MSE={trial.value:.4f}, Params={trial.params}")

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20, callbacks=[print_progress])

print('Best params:', study.best_params)





[I 2025-06-30 00:56:26,850] A new study created in memory with name: no-name-51f1b84a-f425-4538-8e67-69854a31e1f7
[I 2025-06-30 00:59:32,815] Trial 0 finished with value: 452.1406347400334 and parameters: {'n_estimators': 224, 'max_depth': 11, 'min_samples_split': 8, 'min_samples_leaf': 5}. Best is trial 0 with value: 452.1406347400334.


Trial 0: MSE=452.1406, Params={'n_estimators': 224, 'max_depth': 11, 'min_samples_split': 8, 'min_samples_leaf': 5}


[I 2025-06-30 01:00:49,311] Trial 1 finished with value: 497.11547619737644 and parameters: {'n_estimators': 102, 'max_depth': 10, 'min_samples_split': 7, 'min_samples_leaf': 10}. Best is trial 0 with value: 452.1406347400334.


Trial 1: MSE=497.1155, Params={'n_estimators': 102, 'max_depth': 10, 'min_samples_split': 7, 'min_samples_leaf': 10}


[I 2025-06-30 01:09:44,703] Trial 2 finished with value: 284.7507181948592 and parameters: {'n_estimators': 373, 'max_depth': 26, 'min_samples_split': 12, 'min_samples_leaf': 5}. Best is trial 2 with value: 284.7507181948592.


...

Trial 17: MSE=347.7259, Params={'n_estimators': 315, 'max_depth': 15, 'min_samples_split': 5, 'min_samples_leaf': 2}


[I 2025-06-30 02:52:13,688] Trial 18 finished with value: 245.76572220067274 and parameters: {'n_estimators': 392, 'max_depth': 29, 'min_samples_split': 10, 'min_samples_leaf': 4}. Best is trial 18 with value: 245.76572220067274.


Trial 18: MSE=245.7657, Params={'n_estimators': 392, 'max_depth': 29, 'min_samples_split': 10, 'min_samples_leaf': 4}


[I 2025-06-30 03:01:37,221] Trial 19 finished with value: 301.8628295359279 and parameters: {'n_estimators': 410, 'max_depth': 30, 'min_samples_split': 14, 'min_samples_leaf': 7}. Best is trial 18 with value: 245.76572220067274.


Trial 19: MSE=301.8628, Params={'n_estimators': 410, 'max_depth': 30, 'min_samples_split': 14, 'min_samples_leaf': 7}
Best params: {'n_estimators': 392, 'max_depth': 29, 'min_samples_split': 10, 'min_samples_leaf': 4}


In [ ]:
best_params = study.best_params

best_rf = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)

best_rf.fit(X_train, y_train)
pred = best_rf.predict(X_val)

from sklearn.metrics import mean_squared_error, r2_score
print(f"튜닝된 모델 MSE: {mean_squared_error(y_val, pred):.4f}")
print(f"튜닝된 모델 R2: {r2_score(y_val, pred):.4f}")


튜닝된 모델 MSE: 157.5143
튜닝된 모델 R2: 0.7644


1. 데이터증강후 결과(랜덤포래스트)
(1) MSE : 464 -> 157로 크게 감소
 - 오차가 3배이상 줄어든것으로 모델이 예측값을 실제값에 훨씬 더 가깝게 맞추고 있음
(2) R^2(결정계수) : 0.7644 -> 전체 약76%의 변동을 설명 가능


In [ ]:
# 테스트 데이터 로드
test_df = pd.read_csv('/content/drive/MyDrive/Boostup2025/test.csv')
test_df.columns = ['ID', 'Canonical_Smiles']
X_test = np.array([smiles_to_morgan(smi) for smi in test_df['Canonical_Smiles']])

# 3. 예측 수행
test_pred = best_rf.predict(X_test)

# 4. sample_submission.csv 불러오기
submission = pd.read_csv('/content/drive/MyDrive/Boostup2025/sample_submission.csv')

# 5. 예측 결과 삽입
submission['Inhibition'] = test_pred

# 6. 파일 저장
submission.to_csv('/content/drive/MyDrive/Boostup2025/final_submission.csv', index=False)

print("✅ 제출 파일 생성 완료: final_submission.csv")




✅ 제출 파일 생성 완료: final_submission.csv
